<a href="https://colab.research.google.com/github/gabrielle-carvalho/notas-python/blob/main/_home_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências


In [ ]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install Unidecode

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install transformers
!pip install -U scikit-learn
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=773dec12db702a409ee623cf3cc26a14f4dd46815601c99618e8005a13bd180f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install --upgrade transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install nltk

In [ ]:
!pip install torch

# Treinamento GPT-2 (Contexto)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Classe TextDataset
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256):
        self.texts = df['text'].values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], return_tensors='pt', padding='max_length', max_length=self.max_length, truncation=True)
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove dimensões extras
        inputs['labels'] = inputs['input_ids'].clone()
        return inputs

# Expandindo o DataFrame com contextos e perguntas
df = pd.DataFrame({
    "label": ["RoboCup@Home league", "Who can participate?", "The competition", "The scenario", "Finals" ],
    "text": [
    "The RoboCup@Home league aims to develop robotic service and assistance technology with high relevance for future personal home applications. It is the largest annual international competition for autonomous service robots and is part of the RoboCup initiative. A set of benchmark tests is used to evaluate the robots' abilities and performance in a realistic, non-standard home environment. The focus is on the following domains, but is not limited to: Human-Robot Interaction and Cooperation, Navigation and Mapping in dynamic environments, Computer Vision and Object Recognition under natural light conditions, Object Manipulation, Adaptive Behaviors, Behavior Integration, Environmental Intelligence, Standardization and Systems Integration. It is co-located with the RoboCup symposium.",
    "Who can participate? Anyone with an independent robot can participate. The @Home league consists of a few tests and an open challenge to demonstrate your robot's skills. To participate in the open challenge it is necessary to participate in at least one test. The competition takes place in a real-world setting.",
    "The RoboCup@Home competition consists of tests that robots must solve. The tests change over the years to become more advanced and act as a general measure of quality in the desired areas. Performance measurement is based on a score derived from competition rules and assessment by a jury.Test criteria are listed below: include human-robot interaction be socially relevant be application-directed/oriented be scientifically challenging be simple and have self-explanatory rules be interesting to watch.",
    "The scenario. The final setting is the real world itself. To gradually develop possible technologies, a basic home environment is provided as a general setting. In the first years (we started in 2006) it consisted of a living room and a kitchen but soon it also began to involve other areas of daily life, such as a garden/park area, a store, a street or other public places.",
    "Finals. RoboCup@Home ends with the finals where the teams with the highest points can play in the scenario that has been created. Teams are evaluated by a jury made up of roboticists and non-roboticists, such as people from industry, human-machine interaction, industrial design, the public and the press. The classification of the finals determines the winner. In the finals there is less focus on technical issues, as reaching the finals already means that the teams are very good at a technical level."
            ]
})

additional_data_BAHIA = pd.DataFrame({
    "label": ["BAHIA", "BAHIA", "BAHIA" ],
    "text": ["Bahia is a Brazilian state located in the Northeast Region. The population of Bahia is 14.9 million inhabitants, the fourth largest in Brazil. The municipality of Salvador is the state capital and also served as the first Brazilian capital between 1549 and 1763. The state's geography is characterized by plains and depressions, as well as two dominant climate types, tropical and semi-arid. Bahia's economy is the main one in the Northeast and focuses on the tertiary sector and the manufacturing industry.",
             "Bahia climate Two climate types are predominant in Bahia. The first is the semi-arid, which occurs in the central region (with the exception of the highest areas) and part of the west of the state. This climate is characterized by high temperatures and low relative humidity, with irregular precipitation and a dry period generally in the winter months. Rainfall is concentrated in the summer, with an annual average of around 800 mm. The tropical climate occurs mainly in the east of Bahia, also marked by high annual temperatures and a higher level of humidity, especially on the coast. Average annual precipitation varies between 1,200 and 1,600 mm. In some coastal cities, this value can exceed 2,000 mm.",
             "Relief of Bahia Plateaus and depressions are the predominant forms of relief in Bahia, which gives most of the state elevations above 300 meters. The west of Bahia and part of the north are part of the Depression Sertaneja and São Francisco, to the east of which the Plateaus and Serras do Leste-Sudeste extend. The highest terrain in the state is located in this domain, with emphasis on Serra do Espinhaço and Chapada Diamantina. This is where the highest point of Bahia and the Northeast is located: Pico do Barbado, 2,033 meters above sea level. In the eastern range, finally, we have the Coastal Plains and Tablelands."
             ]
})

# Definindo o número de vezes que o DataFrame deve ser replicado
n_copies = 10  # Defina o número de vezes que você deseja replicar o DataFrame

# Concatenando com o DataFrame existente
df = pd.concat([df, additional_data_BAHIA] * n_copies, ignore_index=True)

# Dividir em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2)

# Tokenizador e modelo
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

# Dataset e DataLoader
train_dataset = TextDataset(train_df, tokenizer, max_length=256)  # Aumentado o comprimento máximo
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Otimizador e scheduler
optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=0.05)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 15)

# Treinamento
losses = []
lowest_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()
for epoch in range(15):
    total_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()

        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        del inputs, attention_mask, labels, outputs
        torch.cuda.empty_cache()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Época {epoch + 1}, Perda: {avg_loss}')

    losses.append(avg_loss)

    if avg_loss < lowest_loss:
        lowest_loss = avg_loss
        if lowest_loss <= 0.04:
            model.save_pretrained("fine_tuned_gpt2")
            tokenizer.save_pretrained("fine_tuned_gpt2")

plt.plot(range(1, len(losses) + 1), losses, marker='o', linestyle='-')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.grid(True)
plt.show()


KeyboardInterrupt: 

# Treinamento GPT2 (perguntas)


In [ ]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Classe TextDataset para gerenciar o dataset de textos
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256):
        self.texts = df['text'].values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], return_tensors='pt', padding='max_length', max_length=self.max_length, truncation=True)
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove dimensões extras
        inputs['labels'] = inputs['input_ids'].clone()
        return inputs

# Expandindo o DataFrame com contextos e perguntas
questions = [
    "What is the hero’s name in The Legend of Zelda?",
    "What are the names of the ghosts who chase Pac Man and Ms. Pac Man?",
    "What’s the name of the Mythbusters’ crash test dummy?",
    "What is an Oxford comma?",
    "Who was the captain of the Enterprise in the pilot episode of Star Trek?",
    "What is the symbol for the modulus operator in C?",
    "What function is automatically called at the beginning of a C++ program?",
    "Which computer programming languages was introduced by IBM in 1957?",
    "Who is considered as the first programmer?",
    "Has a robot ever killed a person?",
    "Who was HitchBOT?",
    "Are self-driving cars safe?",
    "Who invented the compiler?",
    "Who created the Python Programming Language?",
    "Is Mark Zuckerberg a robot?",
    "Why did you run away?",
    "What kind of salad do robots like?",
    "What did you ate for lunch?",
    "Why did robots get angry so often?",
    "Why shouldn’t R2D2 be allowed in movies?",
    "What’s your favorite style of music?",
    "What does the acronym SMTP represent?",
    "What does the acronym MPEG represent?",
    "What does the acronym GIMP represent?",
    "What does the acronym GNU represent?",
    "What is the most populous city in Brazil?",
    "Which continent is Brazil located in?",
    "On what day, month and year was Brazil’s independence de- clared?",
    "How many states does Brazil have (with Federal District)?",
    "In what year did the first Brazilian astronaut go to space?",
    "What is the only capital of Brazil crossed by the Equator?",
    "How many time zones are there in Brazil?",
    "In which city is the world’s first urban elevator and what is the name of that elevator?",
    "What is the only biome present in Brazil that is exclusive in the world?",
    "Pampulha Lake is a tourist spot in which Brazilian city?",
    "What is the smallest Brazilian state in territorial extension?",
    "Which capitals in Brazil have the same name as your state?",
    "Where is the Itamaraty Palace located?",
    "What was the first name given to Brazil by the Portuguese?",
    "What is the Newest State in Brazil?",
    "What is the oldest state in Brazil?",
    "What is the capital of Ceará?",
    "What is the capital of Rio Grande do Sul?",
    "What is the capital of Rio Grande do Norte?",
    "What is the capital of Brazil?",
    "What is the capital of Pernambuco?",
    "What is the capital of Pará?",
    "What is the capital of Bahia?",
    "Acarajé is a typical food from which state?",
    "What are the colors of Bahia's flag?",
    "What are some typical foods from Bahia?",
    "What is the most popular rhythm in Bahia, played mainly during the Bahian Carnival?"
]
contexts = [
    "Despite most people’s believes, he’s called Link",
    "Inky, Blinky, Pinky, and Clyde",
    "The Mythbusters’ crash test dummy is called Buster",
    "The hotly contested punctuation before a conjunction in a list",
    "The captain of the Enterprise in the pilot episode was Captain Pike",
    "The percentage symbol is used as modulus operator in C",
    "The main function",
    "Fortran was introduced by IBM in 1957",
    "The first programmer was Ada Lovelace",
    "The first known case of robot homicide occurred in 1981, when a robotic arm crushed a Japanese Kawasaki factory worker",
    "A hitchhiking robot that relied on the kindness of strangers to travel the world and was slain by humans",
    "Yes. Car accidents are product of human misconduct",
    "Grace Hoper. She wrote it in her spare time",
    "Python was invented by Guido van Rossum",
    "Sure. I’ve never seen him drink water",
    "I heard an electric can opener",
    "Salads made with ice-borg lettuce.",
    "I had a byte",
    "People kept pushing our buttons.",
    "He says so many foul words they have to bleep everything he says!",
    "I like electronic and heavy Metal",
    "SMTP stands for Simple Mail Transport Protocol",
    "MPEG stands for Moving Picture Experts Group",
    "GNU Image Manipulation Program",
    "GNU is a recursive acronym meaning GNU is Not Unix",
    "São Paulo is the most populous city in Brazil with 12.03 million residents.",
    "The Brazilian territory is located on the South American continent.",
    "On September 7, 1822, Brazil’s independence was declared.",
    "Currently, Brazil is divided into 26 states and the Federal District, altogether there are 27 federative units.",
    "In March 2006, Pontes became the first Brazilian to go to space.",
    "Macapá is the only Brazilian capital crossed by the Equator line.",
    "Brazil is a country with continental dimensions, in all it has four time zones.",
    "The Lacerda Elevator is a public urban elevator located in Salvador, Brazil.",
    "The Caatinga, characterized by its dry, desert habitat is the only one of Brazil’s biomes found exclusively within the country.",
    "Belo Horizonte",
    "Sergipe",
    "São Paulo and Rio de Janeiro",
    "Brasília",
    "Ilha de vera Cruz",
    "Tocantins",
    "Pernambuco",
    "Fortaleza",
    "Porto alegre",
    "Natal",
    "Brasília",
    "Recife",
    "Belém",
    "Salvador",
    "Bahia",
    "White, red and blue",
    "Caruru, vatapá, abará and acarajé.",
    "Axé or pagode."
]

labels = ["General (simple) questions"] * 21 + \
         ["General (complex) questions"] * 4 + \
         ["Questions about Brazil (simple)"] * 21 + \
         ["Questions about Bahia (simple)"] * 5

# Verifique os comprimentos das listas
print("Comprimento de labels:", len(labels))
print("Comprimento de questions:", len(questions))
print("Comprimento de contexts:", len(contexts))

# Ajuste as listas para o comprimento máximo
max_length = min(len(labels), len(questions), len(contexts))
labels = labels[:max_length]
questions = questions[:max_length]
contexts = contexts[:max_length]

# Criação do DataFrame combinando perguntas e contextos
questions_contexts = pd.DataFrame({
    "label": labels,
    "text": [f"{question} {context}" for question, context in zip(questions, contexts)]
})

# Se não houver um DataFrame existente, use diretamente `questions_contexts`
df = questions_contexts.copy()

# Definindo o número de vezes que o DataFrame deve ser replicado
n_copies = 10  # Defina o número de vezes que você deseja replicar o DataFrame

# Concatenando com o DataFrame existente
df = pd.concat([df] * n_copies, ignore_index=True)

# Dividir em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2)

# Tokenizador e modelo
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

# Dataset e DataLoader
train_dataset = TextDataset(train_df, tokenizer, max_length=256)  # Aumentado o comprimento máximo
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Otimizador e scheduler
optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=0.05)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 15)

# Cronometrar tempo do treinamento
start_time = time.time()

# Treinamento
losses = []
lowest_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()
for epoch in range(15):
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    losses.append(avg_loss)

    # Salvando o melhor modelo
    if avg_loss < lowest_loss:
        lowest_loss = avg_loss
        model.save_pretrained("fine_tuned_gpt2")
        tokenizer.save_pretrained("fine_tuned_gpt2")

    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

end_time = time.time()
# Tempo total de treinamento
total_time = end_time - start_time
hours, rem = divmod(total_time, 3600)
minutes, seconds = divmod(rem, 60)
print(f"Tempo total de treinamento: {int(hours)}h {int(minutes)}min {int(seconds)}s")

# Plotando a função de perda
plt.plot(losses)
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Loss durante o treinamento")
plt.show()

Comprimento de labels: 51
Comprimento de questions: 52
Comprimento de contexts: 52


In [ ]:
!pip install SpeechRecognition
!pip install gTTS
!pip install tkinter
!pip install playsound

In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import speech_recognition as sr
from gtts import gTTS
import os
import playsound

# Função para salvar as respostas das perguntas
def saveResponse(text, file_name):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

FILE_NAME = "respostas_voz.txt"

def listen_microphone():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        log_message("Ajustando o ruído ambiente. Por favor, aguarde...")
        recognizer.adjust_for_ambient_noise(source)
        log_message("Por favor, faça sua pergunta...")
        audio = recognizer.listen(source)

    try:
        log_message("Reconhecendo...")
        root.update_idletasks()  # Atualiza a interface enquanto o reconhecimento ocorre
        return recognizer.recognize_google(audio, language="en")
    except sr.UnknownValueError:
        return "Não consegui entender o áudio."
    except sr.RequestError:
        return "Erro ao se conectar ao serviço de reconhecimento de fala."

def load_model_and_tokenizer(model_name):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_length=500, temperature=0.5, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Remover o texto inicial que pode repetir o prompt
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()

    # Limpar texto gerado para remover caracteres extras
    if generated_text.startswith('?'):
        generated_text = generated_text[1:].strip()

    return generated_text

def activate_microphone():
    prompt = listen_microphone()

    if prompt:
        # Exibir a pergunta no campo de texto
        question_text.config(state=tk.NORMAL)
        question_text.delete("1.0", tk.END)
        question_text.insert(tk.END, prompt)
        question_text.config(state=tk.DISABLED)

        # Carregar modelo e tokenizer ajustado
        model_name = "fine_tuned_gpt2"
        model, tokenizer = load_model_and_tokenizer(model_name)

        # Gerar texto
        generated_text = generate_text(model, tokenizer, prompt)

        # Converter texto gerado para áudio
        tts = gTTS(text=generated_text, lang='en')
        audio_file = "resposta.mp3"
        tts.save(audio_file)
        playsound.playsound(audio_file)
        os.remove(audio_file)

        # Exibir a resposta
        text_output.config(state=tk.NORMAL)
        text_output.delete("1.0", tk.END)
        text_output.insert(tk.END, generated_text)
        text_output.config(state=tk.DISABLED)

        # Salvar as respostas
        saveResponse(f"\nTópico Solicitado: {prompt}", FILE_NAME)
        saveResponse(f"Texto gerado: {generated_text}", FILE_NAME)
    else:
        text_output.config(state=tk.NORMAL)
        text_output.delete("1.0", tk.END)
        text_output.insert(tk.END, "Não foi recebida nenhuma entrada válida do microfone.")
        text_output.config(state=tk.DISABLED)

def on_instructions_button_click():
    instructions = (
        "Este chatbot permite que você faça perguntas por meio do microfone. "
        "Pressione o botão 'Ativar Microfone', fale a sua pergunta, e o chatbot "
        "irá processá-la e fornecer uma resposta. As mensagens de status, como "
        "'Ajustando o ruído ambiente' e 'Reconhecendo...', são exibidas no Log "
        "de Mensagens."
    )
    messagebox.showinfo("Instruções", instructions)

# Função para adicionar mensagens de log na interface
def log_message(message):
    log_text.configure(state=tk.NORMAL)
    log_text.insert(tk.END, message + '\n')
    log_text.configure(state=tk.DISABLED)
    log_text.yview(tk.END)
    root.update_idletasks()  # Atualiza a interface para garantir que o log seja exibido

# Configuração da interface gráfica
root = tk.Tk()
root.title("GPT-2 Question Answering")
root.geometry("800x600")

# Texto de introdução
intro_label = tk.Label(root, text="GPT-2 Resposta Contexto")
intro_label.pack(padx=10, pady=10)

# Botões e Log de Mensagens
button_frame = tk.Frame(root)
button_frame.pack(padx=10, pady=10)
recognize_button = tk.Button(button_frame, text="Ativar Microfone", command=activate_microphone)
recognize_button.grid(row=0, column=0, padx=5)
instructions_button = tk.Button(button_frame, text="Instruções", command=on_instructions_button_click)
instructions_button.grid(row=0, column=1, padx=5)

log_label = tk.Label(root, text="Log de Mensagens:")
log_label.pack(anchor='w', padx=10)
log_text = scrolledtext.ScrolledText(root, height=8, width=80, state=tk.DISABLED)
log_text.pack(padx=10, pady=5)

# Pergunta
question_label = tk.Label(root, text="Pergunta:")
question_label.pack(anchor='w', padx=10)
question_text = scrolledtext.ScrolledText(root, height=5, width=80, state=tk.DISABLED)
question_text.pack(padx=10, pady=5)

# Resposta Gerada
text_output_label = tk.Label(root, text="Texto gerado:")
text_output_label.pack(anchor='w', padx=10)
text_output = scrolledtext.ScrolledText(root, height=10, width=80, state=tk.DISABLED)
text_output.pack(padx=10, pady=5)

root.mainloop()


# Interface GPT


In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import speech_recognition as sr
from gtts import gTTS
import os
import playsound

# Função para salvar as respostas das perguntas
def saveResponse(text, file_name):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

FILE_NAME = "respostas_voz.txt"

def listen_microphone():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        log_message("Ajustando o ruído ambiente. Por favor, aguarde...")
        recognizer.adjust_for_ambient_noise(source)
        log_message("Por favor, faça sua pergunta...")
        audio = recognizer.listen(source)

    try:
        log_message("Reconhecendo...")
        root.update_idletasks()  # Atualiza a interface enquanto o reconhecimento ocorre
        return recognizer.recognize_google(audio, language="en")
    except sr.UnknownValueError:
        return "Não consegui entender o áudio."
    except sr.RequestError:
        return "Erro ao se conectar ao serviço de reconhecimento de fala."

def load_model_and_tokenizer(model_name):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_length=500, temperature=0.5, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Remover o texto inicial que pode repetir o prompt
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()

    # Limpar texto gerado para remover caracteres extras
    if generated_text.startswith('?'):
        generated_text = generated_text[1:].strip()

    return generated_text

def activate_microphone():
    prompt = listen_microphone()

    if prompt:
        # Exibir a pergunta no campo de texto
        question_text.config(state=tk.NORMAL)
        question_text.delete("1.0", tk.END)
        question_text.insert(tk.END, prompt)
        question_text.config(state=tk.DISABLED)

        # Carregar modelo e tokenizer ajustado
        model_name = "fine_tuned_gpt2"
        model, tokenizer = load_model_and_tokenizer(model_name)

        # Gerar texto
        generated_text = generate_text(model, tokenizer, prompt)

        # Converter texto gerado para áudio
        tts = gTTS(text=generated_text, lang='en')
        audio_file = "resposta.mp3"
        tts.save(audio_file)
        playsound.playsound(audio_file)
        os.remove(audio_file)

        # Exibir a resposta
        text_output.config(state=tk.NORMAL)
        text_output.delete("1.0", tk.END)
        text_output.insert(tk.END, generated_text)
        text_output.config(state=tk.DISABLED)

        # Salvar as respostas
        saveResponse(f"\nTópico Solicitado: {prompt}", FILE_NAME)
        saveResponse(f"Texto gerado: {generated_text}", FILE_NAME)
    else:
        text_output.config(state=tk.NORMAL)
        text_output.delete("1.0", tk.END)
        text_output.insert(tk.END, "Não foi recebida nenhuma entrada válida do microfone.")
        text_output.config(state=tk.DISABLED)

def on_instructions_button_click():
    instructions = (
        "Este chatbot permite que você faça perguntas por meio do microfone. "
        "Pressione o botão 'Ativar Microfone', fale a sua pergunta, e o chatbot "
        "irá processá-la e fornecer uma resposta. As mensagens de status, como "
        "'Ajustando o ruído ambiente' e 'Reconhecendo...', são exibidas no Log "
        "de Mensagens."
    )
    messagebox.showinfo("Instruções", instructions)

# Função para adicionar mensagens de log na interface
def log_message(message):
    log_text.configure(state=tk.NORMAL)
    log_text.insert(tk.END, message + '\n')
    log_text.configure(state=tk.DISABLED)
    log_text.yview(tk.END)
    root.update_idletasks()  # Atualiza a interface para garantir que o log seja exibido

# Configuração da interface gráfica
root = tk.Tk()
root.title("GPT-2 Question Answering")
root.geometry("800x600")

# Texto de introdução
intro_label = tk.Label(root, text="GPT-2 Resposta Contexto")
intro_label.pack(padx=10, pady=10)

# Botões e Log de Mensagens
button_frame = tk.Frame(root)
button_frame.pack(padx=10, pady=10)
recognize_button = tk.Button(button_frame, text="Ativar Microfone", command=activate_microphone)
recognize_button.grid(row=0, column=0, padx=5)
instructions_button = tk.Button(button_frame, text="Instruções", command=on_instructions_button_click)
instructions_button.grid(row=0, column=1, padx=5)

log_label = tk.Label(root, text="Log de Mensagens:")
log_label.pack(anchor='w', padx=10)
log_text = scrolledtext.ScrolledText(root, height=8, width=80, state=tk.DISABLED)
log_text.pack(padx=10, pady=5)

# Pergunta
question_label = tk.Label(root, text="Pergunta:")
question_label.pack(anchor='w', padx=10)
question_text = scrolledtext.ScrolledText(root, height=5, width=80, state=tk.DISABLED)
question_text.pack(padx=10, pady=5)

# Resposta Gerada
text_output_label = tk.Label(root, text="Texto gerado:")
text_output_label.pack(anchor='w', padx=10)
text_output = scrolledtext.ScrolledText(root, height=10, width=80, state=tk.DISABLED)
text_output.pack(padx=10, pady=5)

root.mainloop()


# Inferência GPT-2

In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 32.0 MB/s eta 0:00:00


In [ ]:
!pip install gTTS

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import speech_recognition as sr
from gtts import gTTS
import os

# Salvar as respostas das perguntas
def saveResponse(text, file_name):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

FILE_NAME = "respostas_voz.txt"

def listen_microphone():
    # Texto para simular a entrada de áudio
    return "What are Festas Juninas and where are they most commonly celebrated in Brazil?"

def load_model_and_tokenizer(model_name):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_length=500, temperature=0.5, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()
    return generated_text

def main():
    # Escutar o microfone (modificado para texto fixo)
    prompt = listen_microphone()

    if prompt:
        # Carregar modelo e tokenizer ajustado
        model_name = "fine_tuned_gpt2"
        model, tokenizer = load_model_and_tokenizer(model_name)

        # Gerar texto
        generated_text = generate_text(model, tokenizer, prompt)

        # Converter texto gerado para áudio
        tts = gTTS(text=generated_text, lang='en')
        tts.save("resposta.mp3")
        print("Conteúdo de áudio gravado em resposta.mp3")

        # Imprimir os resultados
        print("\nTópico solicitado:", prompt)
        print("\nTexto gerado:", generated_text)

        saveResponse(f"\nTópico Solicitado: {prompt}", FILE_NAME)
        saveResponse(f"Texto gerado: {generated_text}", FILE_NAME)

    else:
        print("Não foi recebida nenhuma entrada válida do microfone.")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Conteúdo de áudio gravado em resposta.mp3

Tópico solicitado: What are Festas Juninas and where are they most commonly celebrated in Brazil?

Texto gerado: Festas are typical celebrations in the month of June in Brazil, with traditional foods such as corn, pamonha, and quentão.


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import speech_recognition as sr
from gtts import gTTS
import os

# Salvar as respostas das perguntas
def saveResponse(text, file_name):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

FILE_NAME = "respostas_voz.txt"

def listen_microphone():
    # Texto para simular a entrada de áudio
    return "Fale sobre Robôs de Serviço"

def load_model_and_tokenizer(model_name):
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_length=500, temperature=0.2, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()
    return generated_text

def main():
    # Escutar o microfone (modificado para texto fixo)
    prompt = listen_microphone()

    if prompt:
        # Carregar modelo e tokenizer ajustado
        model_name = "fine_tuned_gpt2"
        model, tokenizer = load_model_and_tokenizer(model_name)

        # Gerar texto
        generated_text = generate_text(model, tokenizer, prompt)

        # Converter texto gerado para áudio
        tts = gTTS(text=generated_text, lang='pt')
        tts.save("resposta.mp3")
        print("Conteúdo de áudio gravado em resposta.mp3")

        # Imprimir os resultados
        print("\nTópico solicitado:", prompt)
        print("\nTexto gerado:", generated_text)

        saveResponse(f"\nTópico Solicitado: {prompt}", FILE_NAME)
        saveResponse(f"Texto gerado: {generated_text}", FILE_NAME)

    else:
        print("Não foi recebida nenhuma entrada válida do microfone.")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tópico Solicitado: Fale sobre IA
Texto gerado: Fale sobre IA é uma decisões da UNEB. A é um grupo de robótica da computação geral de tarefas da execuço. É umar código projetado para ufide de forma autônoma. Isso inteligência inteligencia e legibilidade e inteligentes intelibes e formada. Ele é que significa que se formas anteriores e experiências de inteliga humanoide. Além inteligo intelilade inteliò não intelisa e humana. E sistemas intelisigibiles e são proporciona, intel


o algoritmo de inferência possibila a realização de perguntas com base do contexto determinado
2 argumentos são passados
contexto e perguntas relacionadas ao contexto
os dados são pré-processados, tokenizados e convertidos em palavras, usando o tokenizador
modelo produz 2 saídas
pontuações de início (start_scores)
probabilidade de cada token no contexto ser o início da resposta
pontuações de fim (end_scores)
probabilidade de cada token no contexto ser o fim da resposta


# Voz com palavra-chave

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import speech_recognition as sr
from gtts import gTTS
import os
import playsound

# Save question and answer in file
def save_response(question, answer, file_name):
    with open(file_name, 'a') as file:
        file.write(f"Question: {question}\n")
        file.write(f"Answer: {answer}\n")
        file.write("\n")

FILE_NAME = "answer.txt"

# Load fine-tuned GPT-2 model and tokenizer
def load_model_and_tokenizer(model_dir):
    model = GPT2LMHeadModel.from_pretrained(model_dir)
    tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
    return model, tokenizer

model_dir = 'fine_tuned_gpt2'
model, tokenizer = load_model_and_tokenizer(model_dir)

def generate_text(model, tokenizer, prompt, max_length=150, temperature=0.7, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True  # Enable sampling to use temperature and top_p
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Remove the initial prompt text if it's repeated in the generated text
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()

    # Clean up text to remove extra characters
    generated_text = generated_text.replace('?', '').strip()

    return generated_text

def wait_for_keyword(keywords, recognizer):
    while True:
        with sr.Microphone() as source:
            print("Adjusting the ambient noise. Please wait...")
            recognizer.adjust_for_ambient_noise(source)
            print("Waiting for Keyword...")
            audio_file = "microphoneOn.mp3"
            playsound.playsound(audio_file)
            audio = recognizer.listen(source)

        try:
            print("Recognizing...")
            text = recognizer.recognize_google(audio, language='en')
            print("You said: ", text)

            if any(keyword.lower() in text.lower() for keyword in keywords):
                print("Keyword detected!")
                return True
            else:
                print("Keyword not detected. Please try again.")

        except sr.UnknownValueError:
            print("I don't understand what you said. Please, repeat.")
        except sr.RequestError as e:
            print(f"Error in the request to the Google API: {e}")

def recognize_speech_from_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting the ambient noise. Please wait...")
        recognizer.adjust_for_ambient_noise(source)

        print("Please ask your question...")
        audio_file = "microphoneOn.mp3"
        playsound.playsound(audio_file)

        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        return recognizer.recognize_google(audio, language="en")
    except sr.UnknownValueError:
        return "I don't understand you"
    except sr.RequestError:
        return "Error connecting to speech recognition service."

def play_response(answer_text):
    if answer_text.strip():
        tts = gTTS(text=answer_text, lang="en")
        audio_file = "answer.mp3"
        tts.save(audio_file)
        playsound.playsound(audio_file)
        os.remove(audio_file)
    else:
        print("Empty response generated. Unable to play audio")

def main():
    while True:
        question = recognize_speech_from_audio()
        print(f"Recognized Question: {question}")

        if "I don't understand you" in question:
            print("Could not understand the question. Please try again.")
            continue

        # Use a more specific prompt to generate a relevant answer
        prompt = f"Answer the following question: {question}"
        answer = generate_text(model, tokenizer, prompt)
        print(f"Generated answer: {answer}")

        save_response(question, answer, FILE_NAME)

        if answer.strip():
            play_response(answer)
        else:
            print("The model did not generate a valid response. Check the prompt and model.")

def listen_microphone():
    keywords = ["Bill", "Hi Bill", "Ok Bill", "ok"]
    recognizer = sr.Recognizer()

    while True:
        if wait_for_keyword(keywords, recognizer):
            main()

if __name__ == "__main__":
    listen_microphone()


# QA BERT

In [ ]:
from tensorboard import notebook
notebook.list()

No known TensorBoard instances running.


In [ ]:
!tensorboard dev upload \
  --logdir "/content/runs" \
  --name "(BertQA2)" \
  --description "BertQA for Robotics" \
  --one_shot

2024-08-14 18:25:58.302300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 18:25:58.337183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 18:25:58.345445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 18:25:58.369047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 18:26:01.729739: W tensorflow/compiler/tf2

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import numpy as np

# Definir hiperparâmetros de treinamento
learning_rate = 3e-05
train_batch_size = 12
eval_batch_size = 8
seed = 42
optimizer_betas = (0.9, 0.999)
optimizer_epsilon = 1e-08
lr_scheduler_type = "linear"
num_epochs = 2.0

# Crie o conjunto de dados de perguntas e respostas sobre Valorant.
# Triplique o conjunto de dados sobre Valorant.
df = pd.DataFrame({
    "Pergunta": [
        "Como funcionam os robôs autônomos na indústria?",
        "Quais são os sensores mais comuns em robótica?",
        "Qual é a história da robótica industrial?",
        "Quais são as estratégias de navegação em robôs móveis?",
        "Qual é o melhor design para um robô de serviço?",
        "Como usar efetivamente a visão computacional em robótica?",
        "Quais são as diferenças entre robôs industriais e robôs colaborativos?",
        "Como funciona o aprendizado de máquina em robótica?",
        "Quais são as habilidades de um robô autônomo?",
        "Como economizar energia em robôs móveis?",
        "Quais são os melhores algoritmos de localização para robôs?",
        "Como prever o comportamento de robôs em ambientes dinâmicos?",
        "Quais são as estratégias para otimizar a produção em robótica industrial?",
        "Qual é a função de um manipulador robótico?",
        "Quais são os melhores atuadores para movimento em robótica?",
        "Como usar os sensores táteis em robótica de forma eficaz?",
        "Quais são as dicas para programar robôs colaborativos?",
        "Como identificar a melhor solução robótica para uma tarefa específica?",
        "Quais são as características de uma interface homem-máquina eficaz em robótica?",
        "Como funciona o sistema de simulação em robótica?"
    ],
    "Resposta": [
        "Os robôs autônomos na indústria operam de forma independente, tomando decisões com base em sensores e algoritmos.",
        "Sensores comuns em robótica incluem câmeras, sensores de proximidade e sensores inerciais.",
        "A história da robótica industrial remonta a meados do século 20, com o surgimento dos primeiros robôs industriais.",
        "Estratégias de navegação em robôs móveis podem envolver SLAM (Simultaneous Localization and Mapping) e planejamento de trajetória.",
        "O design de um robô de serviço depende da aplicação, com considerações de mobilidade, alcance e carga útil.",
        "A visão computacional é essencial para a percepção visual em robótica, permitindo que os robôs entendam o ambiente.",
        "Robôs industriais são usados em ambientes de produção, enquanto robôs colaborativos interagem com humanos de forma segura.",
        "O aprendizado de máquina em robótica permite que os robôs aprendam e melhorem suas habilidades com o tempo.",
        "Habilidades de um robô autônomo incluem navegação, interação com objetos e tomada de decisões baseada em dados.",
        "Economizar energia em robôs móveis envolve otimização de baterias e uso eficiente de recursos.",
        "Algoritmos de localização, como o filtro de partículas, são usados para determinar a posição de um robô em um ambiente.",
        "Prever o comportamento de robôs em ambientes dinâmicos requer modelagem e planejamento avançado.",
        "Estratégias de otimização de produção na robótica industrial visam aumentar a eficiência e a qualidade da produção.",
        "Um manipulador robótico é responsável por realizar tarefas de manipulação, como pegar e mover objetos.",
        "Atuadores, como motores elétricos e pneumáticos, são usados para criar movimento em robôs.",
        "Sensores táteis permitem que os robôs detectem e respondam ao toque e à pressão em suas superfícies.",
        "Programar robôs colaborativos requer considerações de segurança e interface de usuário intuitiva.",
        "Escolher a melhor solução robótica envolve avaliar requisitos, orçamento e aplicação específica.",
        "Uma interface homem-máquina eficaz facilita a interação entre humanos e robôs de forma intuitiva.",
        "Sistemas de simulação em robótica permitem testar algoritmos e comportamento de robôs em ambientes virtuais."
    ],
    "Texto": [
        "Os robôs autônomos são essenciais para a automação industrial e a eficiência de processos.",
        "Sensores desempenham um papel fundamental na percepção do ambiente por parte dos robôs.",
        "A robótica industrial tem evoluído ao longo das décadas, transformando a produção em muitas indústrias.",
        "A navegação precisa é crucial para robôs móveis que operam em ambientes dinâmicos.",
        "O design de robôs de serviço varia de acordo com as necessidades da tarefa a ser executada.",
        "A visão computacional permite que os robôs interpretem informações visuais de seu ambiente.",
        "Robôs colaborativos estão se tornando cada vez mais comuns em ambientes de trabalho compartilhados.",
        "O aprendizado de máquina capacita os robôs a melhorarem suas habilidades com base em experiências passadas.",
        "Robôs autônomos podem executar uma ampla gama de tarefas, desde navegação até reconhecimento de objetos.",
        "A eficiência energética é fundamental para prolongar a autonomia dos robôs móveis.",
        "Algoritmos de localização desempenham um papel fundamental na navegação precisa de robôs.",
        "Prever o comportamento de robôs é importante para a segurança e o desempenho em ambientes dinâmicos.",
        "A otimização da produção é um objetivo-chave na automação industrial com robôs.",
        "Manipuladores robóticos desempenham um papel importante na montagem e na manipulação de objetos.",
        "Atuadores são os motores que conferem movimento e ação aos robôs.",
        "Sensores táteis permitem que os robôs percebam e reajam a interações físicas com o ambiente.",
        "Programar robôs colaborativos envolve garantir que eles possam interagir com segurança com humanos.",
        "Escolher a solução robótica certa é crucial para atender às necessidades específicas da aplicação.",
        "Uma interface homem-máquina eficaz facilita a operação e a colaboração entre humanos e robôs.",
        "A simulação é uma ferramenta valiosa para o desenvolvimento e teste de algoritmos robóticos."
    ]
})

df_extra2 = pd.DataFrame({
    "Pergunta": [
        "Quais são as aplicações de robótica na indústria aeroespacial?",
        "Como os drones estão sendo utilizados em operações de robótica?",
        "Quais são os desafios da robótica em ambientes extremos, como o espaço?",
        "Como a robótica está transformando a logística e a cadeia de suprimentos?",
        "Quais são as implicações da robótica na criação de empregos?",
        "Como os robôs são programados para reconhecimento de voz e linguagem natural?",
        "Quais são as inovações em robótica biomimética?",
        "Como a robótica está sendo aplicada na educação?",
        "Quais são os riscos de segurança associados à automação robótica?",
        "Como os exoesqueletos robóticos estão sendo utilizados em aplicações industriais?",
        "Quais são os avanços em robótica cirúrgica?",
        "Como os robôs são utilizados na manutenção preditiva de equipamentos industriais?",
        "Quais são as tendências em robótica de entretenimento e jogos?",
        "Como a ética na inteligência artificial se relaciona com a robótica autônoma?",
        "Quais são os benefícios da robótica na exploração subterrânea?",
        "Como os robôs colaborativos estão sendo integrados em linhas de produção?",
        "Quais são os desafios da robótica social na interação com seres humanos?",
        "Como a robótica está impactando a fabricação de dispositivos médicos?",
        "Quais são as considerações éticas em robótica militar?",
        "Como a robótica está sendo utilizada na conservação ambiental?"
    ],
    "Resposta": [
        "A robótica na indústria aeroespacial é amplamente utilizada em montagem, inspeção e manutenção de aeronaves.",
        "Drones desempenham papéis cruciais em diversas operações, como monitoramento ambiental, entregas e inspeções.",
        "Ambientes extremos, como o espaço, apresentam desafios únicos, incluindo radiação e condições de microgravidade.",
        "A robótica transforma a logística ao automatizar processos de armazenamento, picking e embalagem em centros de distribuição.",
        "A automação robótica pode criar empregos especializados, mas também levanta preocupações sobre a substituição de empregos tradicionais.",
        "Programação para reconhecimento de voz e linguagem natural envolve algoritmos avançados de processamento de linguagem natural.",
        "A robótica biomimética se inspira na natureza para desenvolver robôs com características semelhantes a animais e organismos.",
        "A robótica na educação inclui kits de robótica para ensinar programação e conceitos de engenharia de forma prática.",
        "Riscos de segurança em automação robótica incluem falhas técnicas, hacking e questões éticas relacionadas à segurança humana.",
        "Exoesqueletos robóticos são usados em indústrias para melhorar a ergonomia e reduzir o risco de lesões musculoesqueléticas.",
        "Avanços em robótica cirúrgica incluem sistemas minimamente invasivos e aprimoramentos em técnicas de precisão.",
        "Robôs na manutenção preditiva monitoram o estado de equipamentos, identificando e corrigindo problemas antes que ocorram falhas.",
        "Tendências em robótica de entretenimento incluem robôs interativos e inovações em realidade virtual e aumentada.",
        "A ética na inteligência artificial e robótica autônoma envolve questões de transparência, responsabilidade e bias algorítmico.",
        "Robótica na exploração subterrânea é crucial para tarefas como mapeamento de minas e inspeção de infraestrutura.",
        "Robôs colaborativos são integrados em linhas de produção para trabalhar lado a lado com humanos, aumentando eficiência.",
        "Desafios da robótica social incluem compreensão emocional, interação natural e considerações éticas na colaboração.",
        "A robótica impacta a fabricação de dispositivos médicos, melhorando precisão em procedimentos e automação em produção.",
        "Considerações éticas em robótica militar envolvem autonomia, proporcionalidade e minimização de danos colaterais.",
        "Robótica na conservação ambiental é utilizada em monitoramento de vida selvagem, limpeza de resíduos e restauração de ecossistemas."
    ],
    "Texto": [
        "Robótica aeroespacial contribui para a eficiência e segurança na manutenção de aeronaves e veículos espaciais.",
        "Drones são ferramentas versáteis em operações, desde inspeções de infraestrutura até entregas de emergência.",
        "Explorar ambientes extremos, como o espaço, exige robôs adaptados para suportar condições únicas e desafios técnicos.",
        "Automatizar a logística com robótica melhora a eficiência, reduzindo erros e acelerando o atendimento a pedidos.",
        "O impacto da automação na criação de empregos é um tópico debatido, com vantagens e desafios a serem considerados.",
        "Reconhecimento de voz em robótica permite interações mais naturais, enquanto algoritmos de linguagem natural melhoram a compreensão.",
        "Biomimética na robótica busca inspiração na natureza para criar soluções eficientes e adaptáveis.",
        "Robótica na educação estimula o aprendizado prático de conceitos STEM (Ciência, Tecnologia, Engenharia e Matemática).",
        "Riscos de segurança em robótica destacam a importância de medidas preventivas e éticas na implementação de sistemas autônomos.",
        "Exoesqueletos industriais suportam trabalhadores, reduzindo fadiga e melhorando a segurança em ambientes laborais.",
        "Robótica cirúrgica avançada permite procedimentos menos invasivos e precisos, melhorando a recuperação do paciente.",
        "Manutenção preditiva com robôs previne falhas, reduzindo o tempo de inatividade e os custos associados à manutenção corretiva.",
        "Entretenimento robótico inova com interações imersivas, promovendo experiências únicas para usuários.",
        "Ética em inteligência artificial é fundamental para garantir decisões justas e transparentes em sistemas autônomos.",
        "Exploração subterrânea com robôs é essencial para acessar áreas de difícil alcance e realizar tarefas específicas.",
        "Robôs colaborativos em linhas de produção combinam eficiência e segurança, permitindo a coexistência com trabalhadores.",
        "Robótica social busca superar desafios na interação robô-humano, priorizando a compreensão emocional e o respeito ético.",
        "Impacto da robótica na fabricação de dispositivos médicos inclui melhorias em precisão e consistência na produção.",
        "Ética em robótica militar envolve considerações sobre o uso responsável de tecnologia em contextos de conflito.",
        "Robótica na conservação ambiental contribui para a preservação da biodiversidade e a restauração de ecossistemas."
    ]
})

# Concatenando os DataFrames
df = pd.concat([df, df_extra2] * 3, ignore_index=True)

# Dividir o conjunto de dados em treinamento e validação
train_df, validation_df = train_test_split(df, test_size=0.5, random_state=seed)

# Importar o modelo DistilBERT e o tokenizer
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")

# Adicionar dropout e camada linear adicional ao modelo
dropout_prob = 0.3
model.distilbert.transformer.layer[-1].output = nn.Sequential(
    nn.Dropout(dropout_prob),
    nn.Linear(model.config.dim, model.config.dim),
    nn.ReLU(),
    nn.Dropout(dropout_prob),
    nn.Linear(model.config.dim, model.config.num_labels),
    nn.Linear(model.config.num_labels, model.config.num_labels)  # Camada adicional
)

# Adicionar função de ativação Softmax no final do modelo
class CustomDistilBERTForQuestionAnswering(DistilBertForQuestionAnswering):
    def forward(self, **inputs):
        outputs = super().forward(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Aplicar função de ativação softmax para obter probabilidades
        start_probs = F.softmax(start_logits, dim=1)
        end_probs = F.softmax(end_logits, dim=1)

        # Retornar probabilidades em vez de logits
        return start_probs, end_probs

# Criar uma instância do modelo personalizado
model = CustomDistilBERTForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

# Definir otimizador e taxa de aprendizado
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=optimizer_betas, eps=optimizer_epsilon)

# Definir o programador de taxa de aprendizado
num_training_steps = len(train_df) * num_epochs
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda step: 1 - step / num_training_steps)

# Listas para armazenar métricas de treinamento e validação
train_losses = []
validation_losses = []
accuracies = []

# Loop de treinamento
for epoch in range(int(num_epochs)):
    model.train()
    total_loss = 0.0

    for i in range(0, len(train_df), train_batch_size):
        batch_df = train_df.iloc[i:i+train_batch_size]

        inputs = tokenizer(batch_df["Pergunta"].tolist(), batch_df["Texto"].tolist(), return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        start_probs = outputs[0]
        end_probs = outputs[1]

        # Calcular a perda com base nas probabilidades de início e fim
        loss = -torch.log(start_probs) - torch.log(end_probs)
        loss = loss.mean()
        total_loss += loss.item()

        # Retropropagação e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    # Calcular a perda média
    average_loss = total_loss / len(train_df)
    train_losses.append(average_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

    # Validar o modelo no conjunto de validação
    model.eval()
    total_accuracy = 0.0

    for i in range(0, len(validation_df), eval_batch_size):
        batch_df = validation_df.iloc[i:i+eval_batch_size]

        inputs = tokenizer(batch_df["Pergunta"].tolist(), batch_df["Texto"].tolist(), return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        start_probs = outputs[0]
        end_probs = outputs[1]

        # Encontrar as posições de início e fim preditas
        start_pred = torch.argmax(start_probs, dim=1)
        end_pred = torch.argmax(end_probs, dim=1)

        # Obter as respostas preditas
        predicted_answers = [tokenizer.decode(input_ids[start:end+1]) for input_ids, start, end in zip(inputs["input_ids"], start_pred, end_pred)]

        # Calcular a acurácia
        true_answers = batch_df["Resposta"].tolist()
        accuracy = sum(1.0 if pred == true else 0.0 for pred, true in zip(predicted_answers, true_answers))
        total_accuracy += accuracy

    # Calcular a acurácia média
    average_accuracy = total_accuracy / len(validation_df)
    accuracies.append(average_accuracy)
    print(f"Validation Accuracy: {average_accuracy:.4f}")

# Salvar o modelo treinado
model.save_pretrained("fine_tuned_distilbert")
# Salvar o tokenizer separadamente
tokenizer.save_pretrained("fine_tuned_distilbert")

Epoch 1/2.0, Loss: 0.8914
Validation Accuracy: 0.0000
Epoch 2/2.0, Loss: 0.7224
Validation Accuracy: 0.0000


('fine_tuned_distilbert/tokenizer_config.json',
 'fine_tuned_distilbert/special_tokens_map.json',
 'fine_tuned_distilbert/vocab.txt',
 'fine_tuned_distilbert/added_tokens.json')

In [ ]:
!pip install datasets transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Inferência QABERT

In [ ]:
!pip install tkinter
!pip install speechrecognition
!pip install gTTS
!pip install playsound

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 47.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=8ceb00eeec5e094bcbd412aabdd17c2f9b4ad3bdbb233f9da56baa2ea36c52da
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [ ]:
import torch
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer

model = DistilBertForQuestionAnswering.from_pretrained("fine_tuned_distilbert")
tokenizer = DistilBertTokenizer.from_pretrained("fine_tuned_distilbert")


def ask_question(context, question, temperature=1.0, min_confidence=0.2, max_retries=5):
    for _ in range(max_retries):
        inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        start_scores = outputs.start_logits / temperature
        end_scores = outputs.end_logits / temperature

        # função softmax aos scores
        start_probs = torch.softmax(start_scores, dim=1)
        end_probs = torch.softmax(end_scores, dim=1)

        # Amostragem ponderada para obter índices de início e fim
        start_index = torch.multinomial(start_probs, 1).item()
        end_index = torch.multinomial(end_probs, 1).item() + 1

        answer = tokenizer.decode(inputs.input_ids[0][start_index:end_index])

        # Verifique se a resposta não está vazia
        if answer.strip():
            return answer

    # Se após várias tentativas não for possível obter uma resposta não vazia, retorne uma mensagem padrão
    return "Não foi possível encontrar uma resposta relevante."


context = ("Bahia is a Brazilian state located in the Northeast Region. The population of Bahia is 14.9 million inhabitants, the fourth largest in Brazil. The municipality of Salvador is the state capital and also served as the first Brazilian capital between 1549 and 1763. The state's geography is characterized by plains and depressions, as well as two dominant climate types, tropical and semi-arid. Bahia's economy is the main one in the Northeast and focuses on the tertiary sector and the manufacturing industry. Bahia climate Two climate types are predominant in Bahia. The first is the semi-arid, which occurs in the central region (with the exception of the highest areas) and part of the west of the state. This climate is characterized by high temperatures and low relative humidity, with irregular precipitation and a dry period generally in the winter months. Rainfall is concentrated in the summer, with an annual average of around 800 mm. The tropical climate occurs mainly in the east of Bahia, also marked by high annual temperatures and a higher level of humidity, especially on the coast. Average annual precipitation varies between 1,200 and 1,600 mm. In some coastal cities, this value can exceed 2,000 mm. Relief of Bahia Plateaus and depressions are the predominant forms of relief in Bahia, which gives most of the state elevations above 300 meters. The west of Bahia and part of the north are part of the Depression Sertaneja and São Francisco, to the east of which the Plateaus and Serras do Leste-Sudeste extend. The highest terrain in the state is located in this domain, with emphasis on Serra do Espinhaço and Chapada Diamantina. This is where the highest point of Bahia and the Northeast is located: Pico do Barbado, 2,033 meters above sea level. In the eastern range, finally, we have the Coastal Plains and Tablelands")

question = "What's the population of Bahia?"



answer = ask_question(context, question, temperature=0.6)
print("Pergunta:", question)
print("\nResposta (Temperatura 0.6):", answer)

answer = ask_question(context, question, temperature=0.8)
print("\nResposta (Temperatura 0.8):", answer)

answer = ask_question(context, question, temperature=1.0)
print("\nResposta (Temperatura 1.0):", answer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: fine_tuned_distilbert is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

# Interface QABERT

In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import torch
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer
import speech_recognition as sr
from gtts import gTTS
import os
import playsound

# Função para salvar a pergunta e a resposta no arquivo
def saveResponse(question, answer, file_name):
    with open(file_name, 'a') as file:
        file.write(f"Pergunta: {question}\n")
        file.write(f"Resposta: {answer}\n")
        file.write("\n")  # Adiciona uma linha em branco para separar as entradas

FILE_NAME = "respostas_voz.txt"

# Carregar o modelo e o tokenizador do DistilBERT
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def generate_answer(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # Obter as pontuações das posições da resposta
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Identificar as posições de início e fim da resposta
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Converter a resposta em texto
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    # Corrigir possíveis problemas com espaços e tokens
    answer = answer.strip()

    return answer

def recognize_speech_from_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        log_message("Ajustando o ruído ambiente. Por favor, aguarde...")
        recognizer.adjust_for_ambient_noise(source)
        log_message("Por favor, faça sua pergunta...")
        audio = recognizer.listen(source)

    try:
        log_message("Reconhecendo...")
        root.update_idletasks()  # Atualiza a interface enquanto o reconhecimento ocorre
        return recognizer.recognize_google(audio, language="en")
    except sr.UnknownValueError:
        return "Não consegui entender o áudio."
    except sr.RequestError:
        return "Erro ao se conectar ao serviço de reconhecimento de fala."

def play_response_audio(answer_text):
    if answer_text.strip():  # Verifica se a resposta não está vazia
        tts = gTTS(text=answer_text, lang='en')
        audio_file = "resposta.mp3"
        tts.save(audio_file)
        playsound.playsound(audio_file)
        os.remove(audio_file)
    else:
        log_message("Resposta vazia gerada. Não é possível reproduzir o áudio.")

def on_recognize_button_click():
    question = recognize_speech_from_audio()
    question_entry.delete(0, tk.END)
    question_entry.insert(0, question)
    answer = generate_answer(context_text.get("1.0", tk.END).strip(), question)

    # Salvar a pergunta e a resposta no histórico
    saveResponse(question, answer, FILE_NAME)
    history_text.configure(state=tk.NORMAL)
    history_text.insert(tk.END, f"Pergunta: {question}\nResposta: {answer}\n\n")
    history_text.configure(state=tk.DISABLED)

    # Reproduzir a resposta em áudio
    play_response_audio(answer)

def on_instructions_button_click():
    instructions = (
        "Este chatbot permite que você faça perguntas por meio do microfone. "
        "Pressione o botão 'Ativar Microfone', fale a sua pergunta, e o chatbot "
        "irá processá-la e fornecer uma resposta. As mensagens de status, como "
        "'Ajustando o ruído ambiente' e 'Reconhecendo...', são exibidas no Log "
        "de Mensagens."
    )
    messagebox.showinfo("Instruções", instructions)

# Função para adicionar mensagens de log na interface
def log_message(message):
    log_text.configure(state=tk.NORMAL)
    log_text.insert(tk.END, message + '\n')
    log_text.configure(state=tk.DISABLED)
    log_text.yview(tk.END)
    root.update_idletasks()  # Atualiza a interface para garantir que o log seja exibido

# Configuração da interface gráfica
root = tk.Tk()
root.title("DistilBert Question Answering")
root.geometry("800x600")

# Texto de introdução
intro_label = tk.Label(root, text="BILL resposta contexto")
intro_label.pack(padx=10, pady=10)

# Contexto
context_label = tk.Label(root, text="Contexto:")
context_label.pack(anchor='w', padx=10)
context_text = scrolledtext.ScrolledText(root, height=6, width=80)
context_text.pack(padx=10, pady=5)
context_text.insert(tk.END, "")

# Pergunta
question_label = tk.Label(root, text="Pergunta:")
question_label.pack(anchor='w', padx=10)
question_entry = tk.Entry(root, width=80)
question_entry.pack(padx=10, pady=5)

# Botões "Ativar Microfone" e "Instruções"
button_frame = tk.Frame(root)
button_frame.pack(padx=10, pady=10)
recognize_button = tk.Button(button_frame, text="Ativar Microfone", command=on_recognize_button_click)
recognize_button.grid(row=0, column=0, padx=5)
instructions_button = tk.Button(button_frame, text="Instruções", command=on_instructions_button_click)
instructions_button.grid(row=0, column=1, padx=5)

# Histórico de Respostas
history_label = tk.Label(root, text="Histórico de Respostas:")
history_label.pack(anchor='w', padx=10)
history_text = scrolledtext.ScrolledText(root, height=4, width=80, state=tk.DISABLED)
history_text.pack(padx=10, pady=5)

# Log de Mensagens
log_label = tk.Label(root, text="Log de Mensagens:")
log_label.pack(anchor='w', padx=10)
log_text = scrolledtext.ScrolledText(root, height=8, width=80, state=tk.DISABLED)
log_text.pack(padx=10, pady=5)

root.mainloop()


# Voz com palavra-chave

In [ ]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer
import torch
import speech_recognition as sr
from gtts import gTTS
import os
import playsound

# Save question and answer in file
def save_response(question, answer, file_name):
    with open(file_name, 'a') as file:
        file.write(f"Question: {question}\n")
        file.write(f"Answer: {answer}\n")
        file.write("\n")

FILE_NAME = "answer.txt"

# Load DistilBERT model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def generate_answer(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"].squeeze().tolist()  # Convert tensor to list, removing singleton dimensions

    # Get the scores of the answer positions
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Identify the start and end positions of the response
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert answer to text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    # Fix possible problems with spaces and tokens
    answer = answer.strip()

    return answer

def wait_for_keyword(keywords, recognizer):
    while True:
        with sr.Microphone() as source:
            print("Adjusting the ambient noise. Please wait...")
            recognizer.adjust_for_ambient_noise(source)
            print("Waiting for Keyword...")
            audio_file = "microphoneOn.mp3"
            playsound.playsound(audio_file)
            audio = recognizer.listen(source)

        try:
            print("Recognizing...")
            text = recognizer.recognize_google(audio, language='en')
            print("You said: ", text)

            if any(keyword.lower() in text.lower() for keyword in keywords):
                print("Keyword detected!")
                return True

        except sr.UnknownValueError:
            print("I don't understand what you said. Please, repeat.")
        except sr.RequestError as e:
            print(f"Error in the request to the Google API: {e}")

def recognize_speech_from_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting the ambient noise. Please wait...")
        recognizer.adjust_for_ambient_noise(source)

        print("Please ask your question...")
        audio_file = "microphoneOn.mp3"
        playsound.playsound(audio_file)

        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        return recognizer.recognize_google(audio, language="en")
    except sr.UnknownValueError:
        return "I don't understand you"
    except sr.RequestError:
        return "Error connecting to speech recognition service."

def play_response(answer_text):
    if answer_text.strip():
        tts = gTTS(text=answer_text, lang="en")
        audio_file = "answer.mp3"
        tts.save(audio_file)
        playsound.playsound(audio_file)
    else:
        print("Empty response generated. Unable to play audio")

def main():
    context = (
    "Tokyo is the capital city of Japan and one of the most populous urban areas in the world. It is known for its modern skyline, historic temples, and vibrant culture. Tokyo was originally a small fishing village named Edo. It became the seat of the Tokugawa shogunate in 1603, and the city was renamed Tokyo, which means 'Eastern Capital.' Today, Tokyo is famous for its bustling districts like Shibuya and Shinjuku, the historic Asakusa district, and landmarks such as the Tokyo Tower and the Imperial Palace. The city also boasts some of the best public transportation systems globally, including its extensive subway network. Tokyo is a hub for technology, fashion, and cuisine, offering a unique blend of traditional and contemporary experiences.\n\n"
    "Economically, Tokyo is a major financial center, often ranking among the world's top cities for business and commerce. It is home to the Tokyo Stock Exchange, one of the largest stock exchanges in the world by market capitalization. The city's economy is diverse, encompassing sectors such as technology, manufacturing, and retail. Tokyo's Shibuya and Ginza districts are renowned for their high-end shopping and entertainment, drawing visitors from all over the globe.\n\n"
    "Culturally, Tokyo is a melting pot of traditional Japanese culture and modern trends. It hosts numerous festivals throughout the year, including the cherry blossom festivals in Ueno Park and the lively Sumida River Fireworks Festival. The city is also known for its thriving arts scene, with many museums, galleries, and theaters. Akihabara, a district in Tokyo, is famous for its electronics shops and is a haven for anime and manga enthusiasts.\n\n"
    "Tokyo is also a global leader in technology and innovation. The city is home to several high-tech industries and research institutions. It is known for its cutting-edge infrastructure, including the Shinkansen (bullet train) network that connects Tokyo with other major cities in Japan. Additionally, Tokyo's urban planning includes impressive skyscrapers, such as the Tokyo Skytree, which offers panoramic views of the city.\n\n"
    "In terms of environment, Tokyo has made significant efforts to address sustainability. The city has various green spaces and parks, such as Yoyogi Park and Shinjuku Gyoen, where residents and visitors can enjoy nature amidst the urban landscape. Tokyo is also investing in eco-friendly technologies and practices to improve air quality and reduce its environmental footprint."
)
    print("Starting the QA system")

    while True:
        question = recognize_speech_from_audio()
        print(f"Recognized Question: {question}")

        answer = generate_answer(context, question)
        print(f"Generated answer: {answer}")

        save_response(question, answer, FILE_NAME)

        play_response(answer)

def listen_microphone():
    keywords = ["Bill", "Hi Bill", "Ok Bill"]
    recognizer = sr.Recognizer()

    while True:
        if wait_for_keyword(keywords, recognizer):
            main()

if __name__ == "__main__":
    listen_microphone()


ModuleNotFoundError: No module named 'speech_recognition'